In [6]:
%run data_gen.ipynb
%run models.ipynb

In [7]:
dg = DataGen(1)

In [8]:
n_batches = int(len(dg.vocab)/100)
batch_size = 8

In [9]:
gen = dg.gen(batch_size=batch_size, n_batches=n_batches, windows_size=4)

In [10]:
main_model, aux_model = conv_model(len(dg.vocab), 44)

In [11]:
history = main_model.fit_generator(gen, steps_per_epoch=n_batches, epochs=60)

Epoch 1/60
147/147 [==============================] - 12s 84ms/step - loss: 15.8886 - acc: 8.5034e-04
Epoch 2/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 3/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 4/60
147/147 [==============================] - 9s 63ms/step - loss: 16.0770 - acc: 0.0026
Epoch 5/60
147/147 [==============================] - 9s 62ms/step - loss: 16.0770 - acc: 0.0026
Epoch 6/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 7/60
147/147 [==============================] - 10s 70ms/step - loss: 16.0770 - acc: 0.0026
Epoch 8/60
147/147 [==============================] - 9s 63ms/step - loss: 16.0770 - acc: 0.0026
Epoch 9/60
147/147 [==============================] - 9s 59ms/step - loss: 16.0770 - acc: 0.0026 0s - loss: 16.0764 - acc: 0.
Epoch 10/60
147/147 [==============================] - 9s 60ms/step - loss: 16.0770 - acc: 0

147/147 [==============================] - 10s 66ms/step - loss: 16.0770 - acc: 0.0026
Epoch 44/60
147/147 [==============================] - 10s 67ms/step - loss: 16.0770 - acc: 0.0026
Epoch 45/60
147/147 [==============================] - 10s 68ms/step - loss: 16.0770 - acc: 0.0026
Epoch 46/60
147/147 [==============================] - 9s 65ms/step - loss: 16.0770 - acc: 0.0026
Epoch 47/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 48/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 49/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 50/60
147/147 [==============================] - 9s 64ms/step - loss: 16.0770 - acc: 0.0026
Epoch 51/60
147/147 [==============================] - 10s 65ms/step - loss: 16.0770 - acc: 0.0026
Epoch 52/60
147/147 [==============================] - 10s 65ms/step - loss: 16.0770 - acc: 0.0026
Epoch 53/60
147/147 [======

In [ ]:
word_embeds = {}
for i in range(30):
    x, y = next(gen)
    embedding = aux_model.predict(x)
    words = dg.one_hot_decode(y)
    for j in range(len(y)):
        word_embeds[words[j]] = embedding[j]



In [ ]:
def euclidean_dist(v, u):
    return np.sqrt(np.sum((v-u) ** 2))
def cosine(u, v):
    return u.dot(v)

In [ ]:
def find_closest(word, embedding):
    min_distance = 1000
    close_words = []
    word_vec = embedding[word]
    for key in embedding.keys():
        vector = embedding[key]
        if euclidean_dist(vector, word_vec) < min_distance and not np.array_equal(vector, word_vec):
            min_distance = euclidean_dist(vector, word_vec)
            close_words.append(key)
    close_words.reverse()
    return close_words

In [ ]:
s = ''
file = open('sim/sigmoid_wr_0pa44d_100_e50_b100_nb40.txt', 'w', encoding='utf-8')
for key in word_embeds:
    s += key + ' : '
    s += ' '.join(find_closest(key, word_embeds)[:5])
    s += '\n'
    file.write(s)
    s = ''

In [ ]:
# u2_word = 'በእርሱ'
# v1 = word_embeds['በእንግሊዝ']
# v2 = word_embeds['ከእንግሊዝ']
# u1 = word_embeds[u2_word]
# v = v1 - v2 + u1
# v_norm = np.linalg.norm(v)
# m_u = -99999
# m_word = None
# for key in word_embeds.keys():
#     u2 = word_embeds[key]
#     d = v.dot(u2)/v_norm
#     if d > m_u and key != u2_word:
#         m_u = d
#         m_word = key
#         print(key)
    
    
# print(m_word)

In [ ]:
%matplotlib notebook
import random
import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
import time

np.random.seed(200)

cho = [np.random.randint(3000) for k in range(100)] 
visualizeWords = [dg.words[i] for i in cho]

visualizeVecs = []
visWords = []
for j in range(len(visualizeWords)):
    key = visualizeWords[j]
    if key in word_embeds:
        visualizeVecs.append(word_embeds[key])
        visWords.append(key)
        
visualizeVecs = visualizeVecs[:50]
visualizeWords = visWords[:50]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeVecs) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])
print(coord.shape)
for i in range(len(visualizeWords)):
#     print(visualizeWords[i])
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

    
plt.rc('font', family='Abyssinica SIL')
plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))
plt.show()
# plt.savefig('images/q3_word_vectors.png')
cho = None
visWords = None
visualizeVecs = None
visualizeWords = None
plt = None